In [1]:
import laspy
import numpy as np
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.cm as cm   
import CSF
from open3d.geometry import KDTreeSearchParamHybrid
from open3d.geometry import PointCloud
import os
import io
import contextlib
import re
import copy
import pyvista as pv
import random


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Visiualized the Pointcloud

In [ ]:
pointcloud_file_path = r'PointClouds\IntermediateFile\non_ground.las'
pointcloud_file_path = r'PointClouds\PartB.las'

In [59]:
# 1. Read the LAS file using laspy.read()
print(f"Attempting to read file: {pointcloud_file_path}")
las = laspy.read(pointcloud_file_path)
print("File read successfully.")

# 2. Extract coordinates (X, Y, Z)
# las.X, las.Y, las.Z provide the scaled (true) coordinates as floats
points = np.vstack((las.x, las.y, las.z)).transpose()
print(f"Extracted {points.shape[0]} points.")

# 3. Create an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# 4. Handle colors (if available in the LAS file)
colors = None
if 'red' in las.point_format.dimension_names and \
    'green' in las.point_format.dimension_names and \
    'blue' in las.point_format.dimension_names:
    colors = np.vstack((las.red, las.green, las.blue)).transpose()
    # Normalize colors from 0-65535 (16-bit) or 0-255 (8-bit) to 0-1 for Open3D
    # if colors.max() > 255: # Check if it's 16-bit color data (common in LAS)
    #     colors = colors / 65535.0
    # else: # Assume 8-bit if max is <= 255, normalize accordingly
    #     colors = colors / 255.0
    pcd.colors = o3d.utility.Vector3dVector(colors)
    print("Point cloud colored using RGB data from LAS file.")
elif 'intensity' in las.point_format.dimension_names:
    intensities = las.intensity.astype(np.float32)

    # Normalize intensities to the range [0, 1]
    # This is crucial for applying a colormap correctly
    if intensities.max() > intensities.min():
        norm_intensities = (intensities - intensities.min()) / (intensities.max() - intensities.min())
    else:
        norm_intensities = np.zeros_like(intensities) # Handle case of uniform intensity

    # Choose a Matplotlib colormap
    # 'jet_r' goes from red (high) -> yellow -> green -> cyan -> blue (low)
    # You can try others: 'viridis', 'plasma', 'inferno', 'magma', 'hot', 'cool', etc.
    # Add '_r' to reverse it (e.g., 'hot_r' for cool colors to hot colors)
    colormap = cm.get_cmap('jet_r')

    # Map the normalized intensities to RGB colors using the colormap
    # colormap returns RGBA, so we take the first 3 channels (RGB)
    colors = colormap(norm_intensities)[:, :3]

    pcd.colors = o3d.utility.Vector3dVector(colors)
    print("Point cloud colored by intensity using a colormap.")
else:
    # Default to a uniform gray color if no specific coloring is available
    pcd.paint_uniform_color([0.7, 0.7, 0.7]) # Light gray color
    print("Point cloud painted with a uniform gray color (no RGB or intensity data found for coloring).")

# --- New part for setting point size with o3d.visualization.draw() ---
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="LAS Point Cloud (Programmatic Size)", width=1024, height=768)

# Create a rendering options object
render_option = vis.get_render_option()

# Set the point size
# A larger number means larger points. Common values are 1, 2, 3, 5, 10.
# Experiment to find what looks best for your data density and scale.
render_option.point_size = 0.5 # Set your desired point size here (float)

# Optionally, you can set background color, etc.
# render_option.background_color = np.asarray([0.1, 0.1, 0.1]) # Dark gray background

vis.add_geometry(pcd)
vis.run() # This starts the visualization loop
vis.destroy_window()
print("Visualization closed.")

Attempting to read file: PointClouds\IntermediateFile\Roof.las


FileNotFoundError: [Errno 2] No such file or directory: 'PointClouds\\IntermediateFile\\Roof.las'

# Visualized the Pointcloud Pyvista

In [ ]:
# ---------- Step 1: Load LAS File ----------
pointcloud_file_path = r'PointClouds\IntermediateFile_B\B_smooth_non_ground.las'
#pointcloud_file_path = r'PointClouds\offset_PartA.las'
las = laspy.read(pointcloud_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'PointClouds\\IntermediateFile_B\\Roof_Clusters.las'

In [6]:
las_x = las.X
las_y = las.Y 
las_z = las.Z 

points = np.vstack((las_x, las_y, las_z)).T.astype(np.float32)
# Extract XYZ coordinates
#points = np.vstack((las.X, las.Y, las.Z)).T.astype(np.float32)

# ---------- Step 2: Normalize Z-values ----------
z = points[:, 2]
z_min, z_max = np.min(z), np.max(z)
z_normalized = (z - z_min) / (z_max - z_min + 1e-8)  # Avoid divide by zero

# ---------- Step 3: Apply Colormap ----------
# Use matplotlib colormap to get RGB colors
cmap = cm.get_cmap("viridis")  # or 'jet', 'plasma', etc.
colors = cmap(z_normalized)[:, :3]  # Drop alpha

# ---------- Step 4: Create PyVista Point Cloud ----------
cloud = pv.PolyData(points)
cloud["colors"] = (colors * 255).astype(np.uint8)  # convert to 0–255

# ---------- Step 5: Visualize ----------
plotter = pv.Plotter()
plotter.add_axes()
plotter.set_background("white")
plotter.add_points(cloud, scalars="colors", rgb=True, point_size=5, render_points_as_spheres=True)
plotter.show(window_size=[1024, 768])

C:\Users\Vettel\AppData\Local\Temp\ipykernel_18912\1443409901.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("viridis")  # or 'jet', 'plasma', etc.


Widget(value='<iframe src="http://localhost:50275/index.html?ui=P_0x2651cc4ca50_1&reconnect=auto" class="pyvis…

In [33]:
# ---------- Load first LAS file ----------
las1 = laspy.read(r"PointClouds\IntermediateFile\smooth_non_ground.las")
points1 = np.vstack((las1.X, las1.Y, las1.Z)).T.astype(np.float32)

cloud1 = pv.PolyData(points1)
cloud1['rgb'] = np.tile([0, 255, 0], (points1.shape[0], 1))  # Red

# ---------- Load second LAS file ----------
las2 = laspy.read(r"PointClouds\IntermediateFile\rough_non_ground.las")
points2 = np.vstack((las2.X, las2.Y, las2.Z)).T.astype(np.float32)

cloud2 = pv.PolyData(points2)
cloud2['rgb'] = np.tile([255, 0, 0], (points2.shape[0], 1))  # Green

# ---------- Plot both clouds ----------
plotter = pv.Plotter()
plotter.add_points(cloud1, scalars='rgb', rgb=True, point_size=3, render_points_as_spheres=True)
plotter.add_points(cloud2, scalars='rgb', rgb=True, point_size=3, render_points_as_spheres=True)
plotter.show(window_size=[1024, 768])

Widget(value='<iframe src="http://localhost:62089/index.html?ui=P_0x18b22210790_14&reconnect=auto" class="pyvi…

# Visialized  las pcd at Same Window

### o3d version

In [31]:
# -------- Load LAS file --------
las_path = r"PointClouds\offset_PartA.las"
las = laspy.read(las_path)
points_las = np.vstack((las.X, las.Y, las.Z)).T.astype(np.float32)

pcd_las = o3d.geometry.PointCloud()
pcd_las.points = o3d.utility.Vector3dVector(points_las)

# Map Z values to colors
z_values = points_las[:, 2]
z_min, z_max = np.min(z_values), np.max(z_values)
normalized_z = (z_values - z_min) / (z_max - z_min + 1e-8)  # Avoid division by zero
print(f"Z values normalized to range [0, 1]: min={normalized_z.min()}, max={normalized_z.max()}")

# Choose colormap (e.g., 'viridis', 'jet', etc.)
cmap = cm.get_cmap('jet')
colors = cmap(normalized_z)[:, :3]  # Drop alpha channel
pcd_las.colors = o3d.utility.Vector3dVector(colors)


# -------- Load PCD file --------
pcd_path = r"PointClouds\IntermediateFile\Roof_clusters.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
# Apply scaling by multiplying XYZ
points = np.asarray(pcd.points)
scaled_points = points * 100.0 + 3.0
pcd.points = o3d.utility.Vector3dVector(scaled_points)
# Set all PCD points to yellow
yellow_color = np.tile([1.0, 1.0, 0.0], (len(pcd.points), 1))
pcd.colors = o3d.utility.Vector3dVector(yellow_color)

Z values normalized to range [0, 1]: min=0.0, max=1.0


C:\Users\Vettel\AppData\Local\Temp\ipykernel_9052\4245619406.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('jet')


In [30]:
# -------- Visualize Together with Point Size Control --------
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="LAS + PCD Visualization", width=1024, height=768)

# Add LAS first (as requested)
vis.add_geometry(pcd_las)
vis.add_geometry(pcd)

render_option = vis.get_render_option()
render_option.point_size = 1.5  # Control point size here
render_option.background_color = np.array([0.1, 0.1, 0.1]) 
# render_option.background_color = np.array([0.1, 0.1, 0.1])  # Optional: dark background

vis.run()
vis.destroy_window()

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


### PyVista Version

In [12]:
# -------- Load LAS file --------
las_path = r"PointClouds\offset_PartA.las"
las = laspy.read(las_path)
points_las = np.vstack((las.X, las.Y, las.Z)).T.astype(np.float32)

# Map Z values to colors (jet colormap)
z_values = points_las[:, 2]
z_min, z_max = np.min(z_values), np.max(z_values)
normalized_z = (z_values - z_min) / (z_max - z_min + 1e-8)
colors_las = (cm.get_cmap('viridis')(normalized_z)[:, :3] * 255).astype(np.uint8)

# Create PyVista point cloud from LAS
cloud_las = pv.PolyData(points_las)
cloud_las["RGB"] = colors_las

# -------- Load PCD file and scale --------
pcd_path = r"PointClouds\IntermediateFile\Roof_clusters.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
points_pcd = np.asarray(pcd.points)
colors_pcd = np.asarray(pcd.colors)
scaled_points = points_pcd * 100.0 + 10.0

# Yellow color for PCD points
#colors_pcd = np.tile([255, 255, 0], (len(scaled_points), 1)).astype(np.uint8)

# Create PyVista point cloud from PCD
cloud_pcd = pv.PolyData(scaled_points)
cloud_pcd['RGB'] = (colors_pcd * 255).astype(np.uint8) 

C:\Users\Vettel\AppData\Local\Temp\ipykernel_18912\4019420654.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors_las = (cm.get_cmap('viridis')(normalized_z)[:, :3] * 255).astype(np.uint8)


In [13]:
# -------- Visualization with PyVista --------
plotter = pv.Plotter(notebook=True)
plotter.add_points(cloud_las, scalars="RGB", rgb=True, point_size=3, name="LAS Points")
plotter.add_points(cloud_pcd, scalars='RGB', rgb=True, point_size=5, name="PCD Points")
plotter.show(title="Interactive 3D View of LAS and PCD", auto_close=False, window_size=[1024, 1024])

Widget(value='<iframe src="http://localhost:50275/index.html?ui=P_0x2650011b4d0_5&reconnect=auto" class="pyvis…

# Add Offset to Origin Data

In [55]:
pointcloud_file_path = r'PointClouds\offset_PartB.las'
#pointcloud_file_path = r'PointClouds\IntermediateFile_B\B_smooth_non_ground.las'

las = laspy.read(pointcloud_file_path)
print("File read successfully.")
# 2. Extract coordinates (X, Y, Z)
# las.X, las.Y, las.Z provide the scaled (true) coordinates as floats
points = np.vstack((las.x, las.y, las.z)).transpose().astype(np.float32)
print(f"Extracted {points.shape[0]} points.")
                                        
x_coords = las.X
y_coords = las.Y
z_coords = las.Z
# 2. Get the range (min and max) of X and Y coordinates
x_min, x_max = np.min(x_coords), np.max(x_coords)
y_min, y_max = np.min(y_coords), np.max(y_coords)

print("\n--- Full Point Cloud XY Range ---")
print(f"X Range: [{x_min:.2f}, {x_max:.2f}]")
print(f"Y Range: [{y_min:.2f}, {y_max:.2f}]")
print("---------------------------------")
print(x_coords[500])

output_file_path = r"PointClouds\offset_PartB.las"

File read successfully.
Extracted 11405824 points.

--- Full Point Cloud XY Range ---
X Range: [0.00, 50000.00]
Y Range: [0.00, 50000.00]
---------------------------------
33092


In [54]:
# Extract X and Y coordinates for range calculation
x_coords = las.x
y_coords = las.y 
z_coords = las.z 

# 2. Get the range (min and max) of X and Y coordinates
x_min, x_max = np.min(x_coords), np.max(x_coords)
y_min, y_max = np.min(y_coords), np.max(y_coords)

print("\n--- Full Point Cloud XY Range ---")
print(f"X Range: [{x_min:.2f}, {x_max:.2f}]")
print(f"Y Range: [{y_min:.2f}, {y_max:.2f}]")
print("---------------------------------")

print(x_coords[20])
offset_x = x_coords - x_min
offset_y = y_coords - y_min
print(offset_x[20])

# Keep Z as is, or you could also apply an offset to Z if desired
offset_z =  z_coords

new_las = laspy.create(point_format=las.header.point_format, 
                           file_version=las.header.version)

# Copy all other dimensions from the original LAS file to the new one
# This ensures that attributes like intensity, classification, RGB colors, etc., are preserved
for dim_name in las.point_format.dimension_names:
    if dim_name not in ['x', 'y', 'z']: # Coordinates are handled separately
        setattr(new_las, dim_name, getattr(las, dim_name))

# Update header min/max values for the new offset data (important for correct visualization and tools)
# laspy 2.0 automatically computes these on write, but explicit setting can be useful
new_las.header.x_min = np.min(offset_x)
new_las.header.x_max = np.max(offset_x)
new_las.header.y_min = np.min(offset_y)
new_las.header.y_max = np.max(offset_y)
new_las.header.z_min = np.min(offset_z) # Z min/max won't change if Z wasn't offset
new_las.header.z_max = np.max(offset_z)
new_las.header.x_offset = 0.0
new_las.header.y_offset = 0.0
new_las.header.z_offset = 0.0
#new_las.header.scales = [0.01, 0.01, 0.01]
new_las.header.scales = [0.01, 0.01, 0.01]
# Set the new offset coordinates
# new_las.X = offset_x
# new_las.Y = offset_y
# new_las.Z = offset_z
new_las.x = offset_x
new_las.y = offset_y
new_las.z = offset_z
print(offset_x[20])

# 5. Write the new LasData object to a new .las file
print(f"\nAttempting to save offset data to: {output_file_path}")
new_las.write(output_file_path)
print(f"Successfully saved offset point cloud to '{output_file_path}'")


--- Full Point Cloud XY Range ---
X Range: [793000.00, 793500.00]
Y Range: [385500.00, 386000.00]
---------------------------------
793322.3948387877
322.39441745949443
322.39441745949443

Attempting to save offset data to: PointClouds\offset_PartB.las
Successfully saved offset point cloud to 'PointClouds\offset_PartB.las'


# Crop the Origin Data

In [11]:
pointcloud_file_path = r'PointClouds\IntermediateFile_B\B_non_ground.las'
#output_file_path = r"PointClouds\cropped_PartB.las" 
current_las = laspy.read(pointcloud_file_path)

In [12]:
point_data = {}
for dim_name in current_las.point_format.dimension_names:
    point_data[dim_name] = getattr(current_las, dim_name)

current_x = point_data['X']
current_y = point_data['Y']
current_z = point_data['Z']

# You can print the min/max of the current data to help set cropping thresholds
print(f"\nCurrent data X Range: [{np.min(current_x):.2f}, {np.max(current_x):.2f}]")
print(f"Current data Y Range: [{np.min(current_y):.2f}, {np.max(current_y):.2f}]")
print(f"Current data Z Range: [{np.min(current_z):.2f}, {np.max(current_z):.2f}]")


# 2. Define the X and Y range for cropping
# IMPORTANT: ADJUST THESE VALUES based on the area you want to crop.
# These thresholds should be defined relative to the current (already offset) coordinates.
# For instance, if your offset data now ranges from 0 to 500 in X and 0 to 600 in Y,
# you might set thresholds like:
crop_x_min = 0.0     # Example: Start from the new origin (0,0)
crop_x_max = 10000.0   # Example: Crop up to 100 meters in X
crop_y_min = 0.0     # Example: Start from the new origin (0,0)
crop_y_max = 10000.0   # Example: Crop up to 100 meters in Y

print(f"\nCropping points where X is between {crop_x_min:.2f} and {crop_x_max:.2f}")
print(f"And Y is between {crop_y_min:.2f} and {crop_y_max:.2f}")

# 3. Crop points based on the defined X and Y ranges
# Use boolean indexing on the current coordinates
x_filter = (current_x >= crop_x_min) & (current_x <= crop_x_max)
y_filter = (current_y >= crop_y_min) & (current_y <= crop_y_max)

# Combine X and Y filters to get the indices of points to keep
cropped_indices = np.where(x_filter & y_filter)[0]

print(f"Number of points after cropping: {len(cropped_indices)}")

if len(cropped_indices) == 0:
    print("No points found within the specified cropping range. Output file will not be created.")
    exit() # Exit if no points are selected to avoid creating empty files

# 4. Create a new LasData object for the cropped points
# Use the header and point format from the current LAS file to maintain metadata
# The header offsets of `current_las` should already be zeroed from the previous step.
new_las = laspy.create(point_format=current_las.header.point_format,
                        file_version=current_las.header.version)

# Copy the header's offsets directly (they should be 0.0 from previous step)
new_las.header.x_offset = current_las.header.x_offset
new_las.header.y_offset = current_las.header.y_offset
new_las.header.z_offset = current_las.header.z_offset

# Copy the scale factors
new_las.header.x_scale = current_las.header.x_scale
new_las.header.y_scale = current_las.header.y_scale
new_las.header.z_scale = current_las.header.z_scale

# Assign the cropped floating-point coordinates
new_las.X = current_x[cropped_indices]
new_las.Y = current_y[cropped_indices]
new_las.Z = current_z[cropped_indices]

# Copy all other dimensions for the cropped points
for dim_name in current_las.point_format.dimension_names:
    # Skip X, Y, Z as they're handled above
    if dim_name not in ['X', 'Y', 'Z']:
        # Use getattr to get the original data for that dimension, then index by cropped_indices
        setattr(new_las, dim_name, getattr(current_las, dim_name)[cropped_indices])

# Update header min/max values for the *cropped* data
# These will reflect the actual min/max of the points that remain after cropping.
new_las.header.x_min = np.min(new_las.X)
new_las.header.x_max = np.max(new_las.X)
new_las.header.y_min = np.min(new_las.Y)
new_las.header.y_max = np.max(new_las.Y)
new_las.header.z_min = np.min(new_las.Z)
new_las.header.z_max = np.max(new_las.Z)

# 5. Write the new LasData object to a new .las file
print(f"\nAttempting to save cropped data to: {output_file_path}")
new_las.write(output_file_path)
print(f"Successfully saved cropped point cloud to '{output_file_path}'")




Current data X Range: [0.00, 50000.00]
Current data Y Range: [0.00, 50000.00]
Current data Z Range: [2500.00, 20486.00]

Cropping points where X is between 0.00 and 10000.00
And Y is between 0.00 and 10000.00
Number of points after cropping: 123793

Attempting to save cropped data to: PointClouds\offset_PartB.las
Successfully saved cropped point cloud to 'PointClouds\offset_PartB.las'


## Crop to 25 Part

In [13]:
# Define the path to your already offset .las file
input_file_path = os.path.join('PointClouds', 'IntermediateFile_B\B_non_ground.las')
# Output directory for the 25 cropped files
output_dir = r"PointClouds\B_non_ground_parts"
os.makedirs(output_dir, exist_ok=True) # Ensure the output directory exists

# 1. Read the already offset LAS file
print(f"Attempting to read offset file: {input_file_path}")
current_las = laspy.read(input_file_path)
print("Offset file read successfully.")

# Get X, Y, Z coordinates and all other dimensions from the current LAS file
point_data = {}
for dim_name in current_las.point_format.dimension_names:
    point_data[dim_name] = getattr(current_las, dim_name)

current_x = point_data['X']
current_y = point_data['Y']
current_z = point_data['Z'] # Z is also needed for the new LAS file

# You can print the min/max of the current data to help verify your understanding
print(f"\nCurrent data X Range: [{np.min(current_x):.2f}, {np.max(current_x):.2f}]")
print(f"Current data Y Range: [{np.min(current_y):.2f}, {np.max(current_y):.2f}]")
print(f"Current data Z Range: [{np.min(current_z):.2f}, {np.max(current_z):.2f}]")

# Define the cropping parameters
x_start_range = 0.0
y_start_range = 0.0
# User stated range 0-49999, so max value is just below 50000
x_end_range = 50000.0 # Define the upper bound for the entire area to be covered
y_end_range = 50000.0 # Define the upper bound for the entire area to be covered

step_size = 10000.0 # Each X and Y step is 10000 units

# Calculate the number of divisions along X and Y
# Add a small epsilon to ensure the upper bound is included if it's exactly on a multiple
num_x_divisions = int(np.ceil((x_end_range - x_start_range) / step_size))
num_y_divisions = int(np.ceil((y_end_range - y_start_range) / step_size))

print(f"\nDividing into {num_x_divisions} x {num_y_divisions} grid sections with a step of {step_size}.")

# Loop through each section (grid cell)
for i in range(num_x_divisions):
    for j in range(num_y_divisions):
        crop_x_min = x_start_range + i * step_size
        crop_x_max = x_start_range + (i + 1) * step_size
        crop_y_min = y_start_range + j * step_size
        crop_y_max = y_start_range + (j + 1) * step_size

        # Ensure max bounds do not exceed the overall end range (important for the last slice)
        crop_x_max = min(crop_x_max, x_end_range)
        crop_y_max = min(crop_y_max, y_end_range)

        print(f"\nProcessing section [{i},{j}]:")
        print(f"  X range: [{crop_x_min:.2f}, {crop_x_max:.2f}]")
        print(f"  Y range: [{crop_y_min:.2f}, {crop_y_max:.2f}]")

        # 2. Crop points based on the defined X and Y ranges for the current section
        x_filter = (current_x >= crop_x_min) & (current_x < crop_x_max) # Use < for upper bound to avoid overlap
        y_filter = (current_y >= crop_y_min) & (current_y < crop_y_max) # Use < for upper bound

        # Combine X and Y filters to get the indices of points to keep in this section
        cropped_indices_section = np.where(x_filter & y_filter)[0]
        
        print(f"  Number of points in this section: {len(cropped_indices_section)}")

        if len(cropped_indices_section) == 0:
            print(f"  Section [{i},{j}] contains no points. Skipping file creation.")
            continue # Skip to the next section if no points are found

        # 3. Create a new LasData object for the cropped points of this section
        new_las_section = laspy.create(point_format=current_las.header.point_format,
                                        file_version=current_las.header.version)

        # Copy the header's offsets directly (they should be 0.0 from previous offset step)
        new_las_section.header.x_offset = current_las.header.x_offset
        new_las_section.header.y_offset = current_las.header.y_offset
        new_las_section.header.z_offset = current_las.header.z_offset

        # Copy the scale factors
        new_las_section.header.x_scale = current_las.header.x_scale
        new_las_section.header.y_scale = current_las.header.y_scale
        new_las_section.header.z_scale = current_las.header.z_scale

        # Assign the cropped floating-point coordinates for this section
        new_las_section.X = current_x[cropped_indices_section]
        new_las_section.Y = current_y[cropped_indices_section]
        new_las_section.Z = current_z[cropped_indices_section]

        # Copy all other dimensions for the cropped points of this section
        for dim_name in current_las.point_format.dimension_names:
            if dim_name not in ['x', 'y', 'z']: # Skip X, Y, Z as they're handled above
                setattr(new_las_section, dim_name, getattr(current_las, dim_name)[cropped_indices_section])

        # Update header min/max values for the *cropped* data of this section
        new_las_section.header.x_min = np.min(new_las_section.X)
        new_las_section.header.x_max = np.max(new_las_section.X)
        new_las_section.header.y_min = np.min(new_las_section.Y)
        new_las_section.header.y_max = np.max(new_las_section.Y)
        new_las_section.header.z_min = np.min(new_las_section.Z)
        new_las_section.header.z_max = np.max(new_las_section.Z)

        # 4. Define the output file name for this section
        #section_output_file_name = f"cropped_part_{i:02d}_{j:02d}.las"
        section_output_file_name = f"non_ground_cropped_part_{i:02d}_{j:02d}.las"
        section_output_file_path = os.path.join(output_dir, section_output_file_name)

        # Write the new LasData object to a new .las file for this section
        print(f"  Saving section to: {section_output_file_path}")
        new_las_section.write(section_output_file_path)
        print(f"  Successfully saved {len(cropped_indices_section)} points for section [{i},{j}].")

Attempting to read offset file: PointClouds\IntermediateFile_B\B_non_ground.las
Offset file read successfully.

Current data X Range: [0.00, 50000.00]
Current data Y Range: [0.00, 50000.00]
Current data Z Range: [2500.00, 20486.00]

Dividing into 5 x 5 grid sections with a step of 10000.0.

Processing section [0,0]:
  X range: [0.00, 10000.00]
  Y range: [0.00, 10000.00]
  Number of points in this section: 123771
  Saving section to: PointClouds\B_non_ground_parts\non_ground_cropped_part_00_00.las
  Successfully saved 123771 points for section [0,0].

Processing section [0,1]:
  X range: [0.00, 10000.00]
  Y range: [10000.00, 20000.00]
  Number of points in this section: 275128
  Saving section to: PointClouds\B_non_ground_parts\non_ground_cropped_part_00_01.las
  Successfully saved 275128 points for section [0,1].

Processing section [0,2]:
  X range: [0.00, 10000.00]
  Y range: [20000.00, 30000.00]
  Number of points in this section: 335288
  Saving section to: PointClouds\B_non_grou

# Cloth Simulate Filter

In [41]:
#pointcloud_file_path = r'PointClouds\cropped_parts\cropped_part_00_01.las'
pointcloud_file_path = r'PointClouds\cropped_PartA.las'
las = laspy.read(pointcloud_file_path)
print("File read successfully.")
# 2. Extract coordinates (X, Y, Z)
# las.X, las.Y, las.Z provide the scaled (true) coordinates as floats
points = np.vstack((las.x, las.y, las.z)).transpose()
print(f"Extracted {points.shape[0]} points.")

File read successfully.
Extracted 378326 points.


In [42]:
csf = CSF.CSF()
csf.params.bSloopSmooth = True        # Set to True for smoother results on steep slopes
csf.params.cloth_resolution = 0.5    # Grid size of cloth. Smaller for finer detail.
csf.params.class_threshold = 0.05      # Max distance from cloth to be classified as ground
csf.params.rigidness = 3              # Rigidity of the cloth (1-3, 1 for soft, 3 for rigid)
csf.params.iterations = 300           # Max iterations for cloth simulation

csf.setPointCloud(points)
# Variables to store indices of ground and non-ground points
ground_indices = CSF.VecInt()
non_ground_indices = CSF.VecInt()

# Perform the filtering
print("Starting CSF filtering...")
csf.do_filtering(ground_indices, non_ground_indices)
print("CSF filtering complete.")

#csf.do_cloth_export()

Starting CSF filtering...
CSF filtering complete.


In [43]:
# Convert VecInt to NumPy arrays for easier indexing
ground_indices_np = np.array(ground_indices)
non_ground_indices_np = np.array(non_ground_indices)

print(f"Number of ground points: {len(ground_indices_np)}")
print(f"Number of non-ground points: {len(non_ground_indices_np)}")

# 3. Create Open3D PointCloud objects for visualization
# Assign different colors for ground and non-ground points

# Ground points (Green)
ground_pcd = o3d.geometry.PointCloud()
if len(ground_indices_np) > 0:
    ground_pcd.points = o3d.utility.Vector3dVector(points[ground_indices_np])
    ground_pcd.paint_uniform_color([0, 1, 0])  # Green for ground points
else:
    print("No ground points found by CSF or an issue occurred during filtering.")

# Non-ground points (Red - for buildings/vegetation)
non_ground_pcd = o3d.geometry.PointCloud()
if len(non_ground_indices_np) > 0:
    non_ground_pcd.points = o3d.utility.Vector3dVector(points[non_ground_indices_np])
    non_ground_pcd.paint_uniform_color([1, 0, 0])  # Red for non-ground points
else:
    print("No non-ground points found by CSF or an issue occurred during filtering.")


# 4. Visualize the point clouds with Open3D
print("Opening Open3D visualization window...")
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Ground (Green) and Non-Ground (Red) Points (CSF Filtered)", width=1024, height=768)

render_option = vis.get_render_option()
render_option.point_size = 0.5 # Adjust point size as desired for visualization
# You can also set a background color if preferred:
# render_option.background_color = np.asarray([0.1, 0.1, 0.1])

if len(ground_indices_np) > 0:
    vis.add_geometry(ground_pcd)
if len(non_ground_indices_np) > 0:
    vis.add_geometry(non_ground_pcd)

vis.run() # This starts the interactive visualization loop
vis.destroy_window()
print("Visualization closed.")

Number of ground points: 236900
Number of non-ground points: 141426
Opening Open3D visualization window...
Visualization closed.


In [45]:
ground_pts = points[list(ground_indices)]
nonground_pts = points[list(non_ground_indices)]

# ---------- Step 3: Create PyVista Point Clouds ----------
cloud_ground = pv.PolyData(ground_pts)
cloud_nonground = pv.PolyData(nonground_pts)

# ---------- Step 4: Visualize ----------
plotter = pv.Plotter()
plotter.add_axes()

# Color the ground brown and non-ground blue
plotter.add_points(cloud_ground, color="green", point_size=5, render_points_as_spheres=True)
plotter.add_points(cloud_nonground, color="red", point_size=5, render_points_as_spheres=True)

plotter.add_legend([
    ("Ground Points", "green"),
    ("Non-Ground Points", "red")
])

plotter.show(window_size=[800, 800])

Widget(value='<iframe src="http://localhost:50275/index.html?ui=P_0x267b2f53790_24&reconnect=auto" class="pyvi…

## Visualized the Cloth

In [ ]:
cloth_file = "cloth_nodes.txt"
cloth_points = np.loadtxt(cloth_file)
print(f"Loaded cloth with {cloth_points.shape[0]} points.")

pointcloud_file_path = r'PointClouds\cropped_parts\cropped_part_00_01.las'
#pointcloud_file_path = r'PointClouds\cropped_partB.las'

las = laspy.read(pointcloud_file_path)  # Replace with your LAS path
las_points = np.vstack((las.X, las.Y, las.Z)).T.astype(np.float32)

Loaded cloth with 1006009 points.


In [13]:
# Create a PyVista point cloud for the LAS file
las_cloud = pv.PolyData(las_points)
las_cloud['Elevation'] = las_points[:, 2]  # Use Z for coloring

# Create a PyVista point cloud for the cloth
cloth_cloud = pv.PolyData(cloth_points)
cloth_cloud['z'] = cloth_points[:, 2]

# Set up the plotter
plotter = pv.Plotter()
plotter.add_mesh(las_cloud, render_points_as_spheres=True, point_size=3, scalars='Elevation', cmap='viridis', name='LAS')
plotter.add_mesh(cloth_cloud, render_points_as_spheres=True, point_size=0.5, color='red', name='Cloth')

plotter.add_axes()

origin = np.array([las.X.min(), las.Y.min(), las.Z.min()])
# Add custom labels near each axis end
plotter.add_point_labels([origin + [1000, 0, 0]], ["X"], font_size=15, text_color='red')
plotter.add_point_labels([origin + [0, 1000, 0]], ["Y"], font_size=15, text_color='green')
plotter.add_point_labels([origin + [0, 0, 1000]], ["Z"], font_size=15, text_color='blue')

plotter.show_grid()
plotter.show()

d:\Anaconda\envs\PointCloud\Lib\site-packages\pyvista\core\utilities\points.py:77: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:62089/index.html?ui=P_0x18a91b72b10_6&reconnect=auto" class="pyvis…

## Save the Non_ground Pointcloud

In [7]:
output_non_ground_file_path = r"PointClouds\non_ground_PartB.las"
#input_file_for_all_dims_path = r'PointClouds\cropped_parts\cropped_part_00_01.las'
input_file_for_all_dims_path = r'PointClouds\cropped_partB.las'


original_las = laspy.read(input_file_for_all_dims_path)
print(f"Read original LAS file again to access all dimensions: {input_file_for_all_dims_path}")

# Extract all XYZ points in the correct format for indexing
xyz_points = np.vstack((original_las.X, original_las.Y, original_las.Z)).transpose()

if 'non_ground_indices_np' not in locals():
    print("WARNING: 'non_ground_indices_np' not found. Using a dummy selection for demonstration.")
    # This is a dummy selection, replace with your actual CSF output
    num_points = original_las.header.number_of_points
    all_indices = np.arange(num_points)
    # Select 50% of points randomly as "non-ground" for demonstration
    non_ground_indices_np = np.random.choice(all_indices, size=int(num_points * 0.5), replace=False)
print(f"Using {len(non_ground_indices_np)} non-ground points for saving.")
# --- END PLACEHOLDER ---

# 1. Select the XYZ coordinates of the non-ground points
non_ground_xyz = xyz_points[non_ground_indices_np]

# 2. Create a new LasData object for only the non-ground points
new_las = laspy.create(point_format=original_las.header.point_format,
                        file_version=original_las.header.version)

# 3. Set the new file's header offsets to 0.0 if you want a local coordinate system
# This ensures that when this new file is read, its points are relative to (0,0,0)
new_las.header.x_offset = 0.0
new_las.header.y_offset = 0.0
new_las.header.z_offset = 0.0

# Keep the original scale factors for consistent precision
new_las.header.x_scale = original_las.header.x_scale
new_las.header.y_scale = original_las.header.y_scale
new_las.header.z_scale = original_las.header.z_scale

# 4. Assign the non-ground X, Y, Z coordinates to the new LasData object
# Remember to use CAPITALIZED X, Y, Z for laspy's scaled coordinates!
new_las.X = non_ground_xyz[:, 0]
new_las.Y = non_ground_xyz[:, 1]
new_las.Z = non_ground_xyz[:, 2]

# 5. Copy all other non-XYZ dimensions for the non-ground points
for dim_name in original_las.point_format.dimension_names:
    if dim_name not in ['x', 'y', 'z']: # Skip X, Y, Z as they are handled above
        # Retrieve the original data for this dimension, then select by non_ground_indices_np
        setattr(new_las, dim_name, getattr(original_las, dim_name)[non_ground_indices_np])

# 6. Update the new header's min/max bounds based on the non-ground data
# This is important for correct display and metadata.
new_las.header.x_min = np.min(new_las.X)
new_las.header.x_max = np.max(new_las.X)
new_las.header.y_min = np.min(new_las.Y)
new_las.header.y_max = np.max(new_las.Y)
new_las.header.z_min = np.min(new_las.Z)
new_las.header.z_max = np.max(new_las.Z)

# 7. Write the new LasData object to the output file
print(f"\nAttempting to save non-ground points to: {output_non_ground_file_path}")
new_las.write(output_non_ground_file_path)
print(f"Successfully saved non-ground point cloud to '{output_non_ground_file_path}'")

Read original LAS file again to access all dimensions: PointClouds\cropped_partB.las
Using 83040 non-ground points for saving.

Attempting to save non-ground points to: PointClouds\non_ground_PartB.las
Successfully saved non-ground point cloud to 'PointClouds\non_ground_PartB.las'


# Calcualte Roughness

In [16]:
input_non_ground_file_path = r"PointClouds\non_ground_PartA.las" 
input_non_ground_file_path = r"PointClouds\non_ground_PartB.las"

In [17]:
# 1. Read the non_ground LAS file
print(f"Attempting to read non-ground point cloud: {input_non_ground_file_path}")
las_data = laspy.read(input_non_ground_file_path)
print("Non-ground file read successfully.")

# Get X, Y, Z coordinates
xyz_points = np.vstack((las_data.X, las_data.Y, las_data.Z)).transpose()
print(f"Loaded {xyz_points.shape[0]} non-ground points.")

if xyz_points.shape[0] == 0:
    print("No points found in the non-ground file. Cannot calculate roughness. Exiting.")
    exit()

# Create an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_points)

# Optional: If colors exist, copy them for initial visualization or later use
if 'red' in las_data.point_format.dimension_names and \
    'green' in las_data.point_format.dimension_names and \
    'blue' in las_data.point_format.dimension_names:
    colors = np.vstack((las_data.red, las_data.green, las_data.blue)).transpose()
    # Normalize colors to [0, 1] if they are 16-bit (0-65535) or 8-bit (0-255)
    if colors.max() > 255: colors = colors / 65535.0
    else: colors = colors / 255.0
    pcd.colors = o3d.utility.Vector3dVector(colors)
    print("Using original RGB colors from non-ground file.")
else:
    pcd.paint_uniform_color([0.7, 0.7, 0.7]) # Default to light gray
    print("No RGB data in non-ground file. Coloring uniformly.")

# 2. Calculate roughness for each point
print("Calculating roughness for each non-ground point...")
# Use a KDTree for efficient nearest neighbor search
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# Number of neighbors to consider for roughness calculation
# Adjust this parameter (k_neighbors) based on your point cloud density
# A larger k will give a smoother estimate, a smaller k will capture finer detail.
k_neighbors =  40

roughness_values = np.zeros(xyz_points.shape[0])

for i in range(xyz_points.shape[0]):
    # Find k nearest neighbors for point i
    # search_knn_vector_3d returns [k, list_of_indices, list_of_distances]
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[i], k_neighbors)
    
    # Get the coordinates of the neighbors
    neighbors = np.asarray(pcd.points)[idx, :]

    # Compute the covariance matrix of the neighbors
    # Need at least 2 points for covariance, ideally more than 3 for a robust plane estimate
    if neighbors.shape[0] >= 3: 
        cov_matrix = np.cov(neighbors, rowvar=False) # rowvar=False means columns are variables (X,Y,Z)
        
        # Perform eigenvalue decomposition on the covariance matrix
        # eigvalsh is for symmetric matrices, which covariance matrices are.
        eigenvalues = np.linalg.eigvalsh(cov_matrix)
        eigenvalues = np.sort(eigenvalues)[::-1] # Sort in descending order: lambda_1 >= lambda_2 >= lambda_3

        # Roughness metric: ratio of the smallest eigenvalue to the sum of eigenvalues
        # lambda_3 represents the variance perpendicular to the local plane.
        # A smaller value means the points are more planar (smoother).
        sum_eigenvalues = np.sum(eigenvalues)
        if sum_eigenvalues > 1e-9: # Avoid division by zero for perfectly flat or identical points
            roughness = eigenvalues[2] / sum_eigenvalues # roughness = lambda_3 / (lambda_1 + lambda_2 + lambda_3)
        else:
            roughness = 0.0 # Handle cases with no variance (e.g., all neighbors are the same point)

        roughness_values[i] = roughness
    else:
        # If not enough neighbors for a robust covariance (e.g., edge cases)
        roughness_values[i] = 0.0 

print("Roughness calculation complete.")

Attempting to read non-ground point cloud: PointClouds\non_ground_PartB.las
Non-ground file read successfully.
Loaded 335288 non-ground points.
No RGB data in non-ground file. Coloring uniformly.
Calculating roughness for each non-ground point...
Roughness calculation complete.


In [20]:
# You can print min/max roughness to help set the threshold
print(f"\nRoughness values range: [{np.min(roughness_values):.6f}, {np.max(roughness_values):.6f}]")

# 3. Define a roughness threshold to divide points
# IMPORTANT: Adjust this 'roughness_threshold' value!
# This threshold determines what you consider "smooth" vs. "rough".
# Experiment with values based on the printed range above and visual inspection of your data.
# A smaller threshold means more points will be classified as "rough".
roughness_threshold = 0.008 # Example threshold: Tune this!

print(f"\nDividing points with a roughness threshold of: {roughness_threshold:.6f}")

# Divide points into two sets based on the threshold
smooth_indices = np.where(roughness_values <= roughness_threshold)[0]
rough_indices = np.where(roughness_values > roughness_threshold)[0]

print(f"Number of 'smooth' non-ground points: {len(smooth_indices)}")
print(f"Number of 'rough' non-ground points: {len(rough_indices)}")

# 4. Apply different colors for visualization
smooth_pcd = pcd.select_by_index(list(smooth_indices))
rough_pcd = pcd.select_by_index(list(rough_indices))

smooth_pcd.paint_uniform_color([0, 0, 1]) # Blue for "smooth" non-ground points
rough_pcd.paint_uniform_color([1, 0, 0])  # Red for "rough" non-ground points

# 5. Visualize the point clouds
print("Opening Open3D visualization window...")
vis = o3d.visualization.Visualizer()
vis.create_window(window_name=f"Non-Ground Roughness: Smooth (Blue) vs Rough (Red) (Threshold={roughness_threshold:.4f})", 
                    width=1024, height=768)

render_option = vis.get_render_option()
render_option.point_size = 1.0 # Adjust point size as desired for visualization
# render_option.background_color = np.asarray([0.1, 0.1, 0.1]) # Dark background

# Add geometries only if they contain points
if len(smooth_indices) > 0:
    vis.add_geometry(smooth_pcd)
if len(rough_indices) > 0:
    vis.add_geometry(rough_pcd)

vis.run()
vis.destroy_window()
print("Visualization closed.")


Roughness values range: [0.000033, 0.310509]

Dividing points with a roughness threshold of: 0.008000
Number of 'smooth' non-ground points: 130273
Number of 'rough' non-ground points: 205015
Opening Open3D visualization window...
Visualization closed.


### Visualized the Roughness Value

In [21]:
las = laspy.read(input_non_ground_file_path)  # Replace with your LAS path
las_points = np.vstack((las.X, las.Y, las.Z)).T.astype(np.float32)

# --- Normalize roughness for color mapping ---
rough_min = np.min(roughness_values)
rough_max = np.max(roughness_values)
rough_norm = (roughness_values - rough_min) / (rough_max - rough_min + 1e-8)

# --- Apply colormap (e.g., viridis, jet) ---
cmap = cm.get_cmap('jet')
colors = cmap(rough_norm)[:, :3]  # Remove alpha
colors = (colors * 255).astype(np.uint8)  # Convert to 0–255 for RGB

# --- Create PyVista point cloud ---
cloud = pv.PolyData(las_points)
cloud['Roughness'] = roughness_values  # Store scalar roughness
cloud['colors'] = colors  # RGB values

# --- Visualize with PyVista ---
plotter = pv.Plotter()
plotter.add_axes()
plotter.set_background("white")

# Option 1: Use RGB directly
plotter.add_points(cloud, scalars='colors', rgb=True, point_size=4)

# Option 2: Use scalar mapping (colorbar shows roughness)
# plotter.add_points(cloud, scalars='Roughness', cmap='viridis', point_size=4, render_points_as_spheres=True)

plotter.add_scalar_bar(title="Roughness", n_labels=5, label_font_size=12)
plotter.show(title="Roughness Visualization", window_size=[1024, 768])

C:\Users\Vettel\AppData\Local\Temp\ipykernel_9052\2432821284.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('jet')


Widget(value='<iframe src="http://localhost:53082/index.html?ui=P_0x1e0312b1c50_1&reconnect=auto" class="pyvis…

In [22]:
# Threshold
roughness_threshold = 0.008

# Split into mask for smooth and rough
smooth_mask = roughness_values <= roughness_threshold
rough_mask = roughness_values > roughness_threshold

# Assign colors
# Blue for smooth, Red for rough
colors = np.zeros((len(las_points), 3), dtype=np.uint8)
colors[smooth_mask] = [0, 0, 255]   # Blue
colors[rough_mask] = [255, 0, 0]    # Red

# Create PyVista PolyData
cloud = pv.PolyData(las_points)
cloud['colors'] = colors

# Plot
plotter = pv.Plotter()
plotter.set_background("white")
plotter.add_axes()

# Add points with RGB colors
plotter.add_points(cloud, scalars='colors', rgb=True, point_size=4)
plotter.show(title="Smooth (Blue) vs Rough (Red) Points", window_size=[1024, 768])

Widget(value='<iframe src="http://localhost:53082/index.html?ui=P_0x1e031337d10_2&reconnect=auto" class="pyvis…

In [23]:
output_smooth_file_path = r"PointClouds\smooth_non_ground_PartB.las"
input_file_for_smooth_points_path = r"PointClouds\non_ground_PartB.las"

las_data_source = laspy.read(input_file_for_smooth_points_path)
print(f"Read source LAS file for smooth points: {input_file_for_smooth_points_path}")

# --- PLACEHOLDER FOR ROUGHNESS CALCULATION RESULT ---
# In a full script, the 'smooth_indices' would be generated by your roughness calculation code.
# For standalone demonstration, let's create some dummy indices.
if 'smooth_indices' not in locals():
    print("WARNING: 'smooth_indices' not found. Using a dummy selection for demonstration.")
    num_points_source = las_data_source.header.number_of_points
    # Dummy: Select 20% of points randomly as "smooth" for demonstration
    smooth_indices = np.random.choice(np.arange(num_points_source), 
                                        size=int(num_points_source * 0.2), 
                                        replace=False)
    smooth_indices.sort() # Sorting is good practice but not strictly necessary for indexing
print(f"Using {len(smooth_indices)} 'smooth' points for saving.")
# --- END PLACEHOLDER ---

if len(smooth_indices) == 0:
    print("No smooth points found based on the threshold. No file will be created.")
    exit() # Exit if no smooth points are selected

# 1. Create a new LasData object for only the smooth points
# Use the header and point format from the source LAS file to maintain metadata structure
new_las = laspy.create(point_format=las_data_source.header.point_format,
                        file_version=las_data_source.header.version)

# 2. Set the new file's header offsets to 0.0 (or copy from source if source was already zeroed)
# This ensures consistency if the non_ground_points.las file already had its offsets at 0.0
new_las.header.x_offset = las_data_source.header.x_offset
new_las.header.y_offset = las_data_source.header.y_offset
new_las.header.z_offset = las_data_source.header.z_offset

# Copy the scale factors from the source
new_las.header.x_scale = las_data_source.header.x_scale
new_las.header.y_scale = las_data_source.header.y_scale
new_las.header.z_scale = las_data_source.header.z_scale

# 3. Assign the smooth X, Y, Z coordinates to the new LasData object
# Remember to use CAPITALIZED X, Y, Z for laspy's scaled coordinates!
new_las.X = las_data_source.X[smooth_indices]
new_las.Y = las_data_source.Y[smooth_indices]
new_las.Z = las_data_source.Z[smooth_indices]

# 4. Copy all other non-XYZ dimensions for the smooth points
for dim_name in las_data_source.point_format.dimension_names:
    if dim_name not in ['x', 'y', 'z']: # Skip X, Y, Z as they are handled above
        # Retrieve the original data for this dimension, then select by smooth_indices
        setattr(new_las, dim_name, getattr(las_data_source, dim_name)[smooth_indices])

# 5. Update the new header's min/max bounds based on the *actual* smooth data's range
new_las.header.x_min = np.min(new_las.X)
new_las.header.x_max = np.max(new_las.X)
new_las.header.y_min = np.min(new_las.Y)
new_las.header.y_max = np.max(new_las.Y)
new_las.header.z_min = np.min(new_las.Z)
new_las.header.z_max = np.max(new_las.Z)

# 6. Write the new LasData object to the output file
print(f"\nAttempting to save 'smooth' points to: {output_smooth_file_path}")
new_las.write(output_smooth_file_path)
print(f"Successfully saved 'smooth' point cloud to '{output_smooth_file_path}'")

Read source LAS file for smooth points: PointClouds\non_ground_PartB.las
Using 130273 'smooth' points for saving.

Attempting to save 'smooth' points to: PointClouds\smooth_non_ground_PartB.las
Successfully saved 'smooth' point cloud to 'PointClouds\smooth_non_ground_PartB.las'


# DBSCAN

In [60]:
input_file_path = r'PointClouds\smooth_non_ground_PartA.las'

# 1. Read the LAS file
print(f"Attempting to read file: {input_file_path}")
las_data = laspy.read(input_file_path)
print("File read successfully.")

# Get X, Y, Z coordinates
xyz_points = np.vstack((las_data.X, las_data.Y, las_data.Z)).transpose()
print(f"Loaded {xyz_points.shape[0]} points.")

if xyz_points.shape[0] == 0:
    print("The input file contains no points. Cannot proceed with analysis. Exiting.")
    exit()

# Create an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_points)

# 2. Perform Connected Component Analysis (using DBSCAN clustering as an approximation)
# DBSCAN clusters points based on density. Points that are close to each other are grouped.
print("\nPerforming DBSCAN clustering for connected components...")

# DBSCAN parameters:
# 'eps_distance': The maximum distance between two points for them to be considered neighbors.
#                 This is critical for defining "connectedness". Tune this based on your point cloud's density.
# 'min_cluster_points': The minimum number of points required to form a dense region (a cluster).
#                 Smaller values yield more, smaller clusters.

# You will likely need to tune these parameters for your specific data and scale!
eps_distance = 100       # Example: 1.0 unit (e.g., meter) search radius
min_cluster_points = 15  # Example: A cluster needs at least 10 points

labels = np.array(pcd.cluster_dbscan(eps=eps_distance, min_points=min_cluster_points, print_progress=True))

max_label = labels.max()
# A label of -1 indicates noise points (points not assigned to any cluster)
num_valid_clusters = max_label + 1 if max_label >= 0 else 0
print(f"DBSCAN found {num_valid_clusters} clusters (components) and {np.sum(labels == -1)} noise points.")

# 3. Filter Components by Point Number Threshold for visualization
# Define the threshold for component size: components smaller than this will be colored gray.
min_component_size_threshold = 15 # Example: Components with fewer than 50 points will be grayed out

print(f"Components with fewer than {min_component_size_threshold} points will be colored gray.")

# Create an array to store colors for all points, initialized to gray
colors = np.full((xyz_points.shape[0], 3), 0.5) # Initialize all points to gray (RGB 0.5, 0.5, 0.5)

# Prepare for distinct coloring of large components using a colormap
# 'tab20' is a good categorical colormap for up to 20 distinct colors.
# If you expect more, consider 'tab20b', 'tab20c', or 'hsv'.
cmap = cm.get_cmap('tab20', num_valid_clusters) 

# Iterate through each unique cluster label
for label in np.unique(labels):
    component_indices = np.where(labels == label)[0]
    component_size = len(component_indices)

    if label == -1:
        # Noise points are already gray from initialization, or you can assign a specific noise color
        print(f"  Noise points (-1 label): {component_size} points (will be gray)")
    elif component_size < min_component_size_threshold:
        # Small component, will remain gray (as per initialization)
        print(f"  Cluster {label}: {component_size} points (too small, will be gray)")
    else:
        # Large component, assign a distinct color from the colormap
        colors[component_indices] = cmap(label)[:3] # Get RGB from colormap, discarding alpha
        print(f"  Cluster {label}: {component_size} points (will be distinctly colored)")

pcd.colors = o3d.utility.Vector3dVector(colors)

# 4. Visualize the point cloud with component colors
print("\nOpening Open3D visualization window for clustered components...")
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Point Cloud Components (Large: Color, Small/Noise: Gray)", 
                    width=1024, height=768)

render_option = vis.get_render_option()
render_option.point_size = 8.0 # Adjust point size for better visibility
# render_option.background_color = np.asarray([0.1, 0.1, 0.1]) # Optional: Dark background

vis.add_geometry(pcd) # Add the point cloud with its assigned colors

vis.run() # Starts the interactive visualization loop
vis.destroy_window() # Closes the window when the user closes it
print("Visualization closed.")

Attempting to read file: PointClouds\smooth_non_ground_PartA.las
File read successfully.
Loaded 94694 points.

Performing DBSCAN clustering for connected components...
DBSCAN found 182 clusters (components) and 2625 noise points.
Components with fewer than 15 points will be colored gray.
  Noise points (-1 label): 2625 points (will be gray)
  Cluster 0: 4671 points (will be distinctly colored)
  Cluster 1: 63 points (will be distinctly colored)
  Cluster 2: 34 points (will be distinctly colored)
  Cluster 3: 6634 points (will be distinctly colored)
  Cluster 4: 193 points (will be distinctly colored)
  Cluster 5: 1308 points (will be distinctly colored)
  Cluster 6: 3237 points (will be distinctly colored)
  Cluster 7: 3009 points (will be distinctly colored)
  Cluster 8: 21 points (will be distinctly colored)
  Cluster 9: 382 points (will be distinctly colored)
  Cluster 10: 24 points (will be distinctly colored)
  Cluster 11: 17741 points (will be distinctly colored)
  Cluster 12: 91

C:\Users\Vettel\AppData\Local\Temp\ipykernel_18912\2075394359.py:53: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', num_valid_clusters)


Visualization closed.


In [104]:
output_file_path = r'PointClouds\clustered_PartA.las'

clustered_indices = np.where(labels != -1)[0]
print(f"\nFound {len(clustered_indices)} points belonging to clusters (excluding noise).")

if len(clustered_indices) > 0:
    # Create a new LasData object for only the clustered points
    new_las = laspy.create(point_format=las_data.header.point_format,
                            file_version=las_data.header.version)

    # Assign X, Y, Z coordinates for the clustered points
    new_las.X = las_data.X[clustered_indices]
    new_las.Y = las_data.Y[clustered_indices]
    new_las.Z = las_data.Z[clustered_indices]

    # Copy all other dimensions for the clustered points
    for dim_name in las_data.point_format.dimension_names:
        if dim_name not in ['x', 'y', 'z']:
            # Use getattr and setattr to dynamically access and set dimensions
            setattr(new_las, dim_name, getattr(las_data, dim_name)[clustered_indices])

    # Update the new header's offsets and scale factors from the original
    new_las.header.x_offset = las_data.header.x_offset
    new_las.header.y_offset = las_data.header.y_offset
    new_las.header.z_offset = las_data.header.z_offset
    new_las.header.x_scale = las_data.header.x_scale
    new_las.header.y_scale = las_data.header.y_scale
    new_las.header.z_scale = las_data.header.z_scale

    # Update the new header's min/max bounds based on the actual clustered data
    new_las.header.x_min = np.min(new_las.X)
    new_las.header.x_max = np.max(new_las.X)
    new_las.header.y_min = np.min(new_las.Y)
    new_las.header.y_max = np.max(new_las.Y)
    new_las.header.z_min = np.min(new_las.Z)
    new_las.header.z_max = np.max(new_las.Z)

    # Write the new LasData object to the output file
    print(f"Saving clustered (non-noise) points to: {output_file_path}")
    new_las.write(output_file_path)
    print(f"Successfully saved {len(clustered_indices)} points to '{output_file_path}'")
else:
    print("No points were identified as belonging to clusters. No file saved.")


Found 48931 points belonging to clusters (excluding noise).
Saving clustered (non-noise) points to: PointClouds\clustered_PartA.las
Successfully saved 48931 points to 'PointClouds\clustered_PartA.las'


# RANSAC

In [2]:
input_file_path = r'PointClouds\smooth_non_ground_PartA.las'
# Load .las file using laspy
las = laspy.read(input_file_path)
points = np.vstack((las.X, las.Y, las.Z)).transpose()
print(las.X.max())

9999


In [ ]:
# Step 1: Prepare Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Optional: estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=10))

max_planes = 20
min_points_per_plane = 30
distance_threshold = 100
segments = []
colors = []
rest = pcd  # Start with full cloud

for i in range(max_planes):
    if len(rest.points) < min_points_per_plane or len(rest.points) < 3:
        break

    # RANSAC plane detection
    plane_model, inliers = rest.segment_plane(
        distance_threshold=distance_threshold,
        ransac_n=5,
        num_iterations=1000
    )

    if len(inliers) < min_points_per_plane:
        break

    plane_cloud = rest.select_by_index(inliers)
    color = [random.random(), random.random(), random.random()]
    segments.append(np.asarray(plane_cloud.points))
    colors.append(np.tile(color, (len(plane_cloud.points), 1)))

    rest = rest.select_by_index(inliers, invert=True)

# Add remaining points as gray
segments.append(np.asarray(rest.points))
colors.append(np.tile([0.5, 0.5, 0.5], (len(rest.points), 1)))

# Combine all segments
all_points = np.vstack(segments)
all_colors = np.vstack(colors)

# Step 2: Visualize with PyVista
cloud = pv.PolyData(all_points)
cloud["colors"] = all_colors

plotter = pv.Plotter()
plotter.add_mesh(cloud, scalars="colors", rgb=True, point_size=5, render_points_as_spheres=True)
plotter.add_axes()
plotter.show(window_size=[800, 800])

Widget(value='<iframe src="http://localhost:60796/index.html?ui=P_0x2ef05d0db50_0&reconnect=auto" class="pyvis…

In [39]:
output_path = r'region_growing_colored_B.pcd'
# Combine all segments into a single point cloud
combined = o3d.geometry.PointCloud()
for seg in segments:
    combined += seg

# Save to PCD file with color
o3d.io.write_point_cloud(output_path, combined)
print(f"Saved segmented point cloud with color to: {output_path}")

Saved segmented point cloud with color to: region_growing_colored_B.pcd


# Remove Point by Z-value

In [40]:
# --- Step 1: Load LAS file ---
input_path = r'PointClouds\IntermediateFile_B\B_non_ground.las'
input_path = r'PointClouds\cropped_PartA.las'
las = laspy.read(input_path)

# --- Step 2: Extract and filter points ---
x, y, z = las.X, las.Y, las.Z
points = np.vstack((x, y, z)).T

# Set threshold
z_threshold = 2500
mask_above = z >= z_threshold
mask_below = z < z_threshold

# Combine mask to match original points
color_array = np.zeros((len(z), 3))  # Initialize all black

# Red for points above threshold
color_array[mask_above] = [1.0, 0.0, 0.0]  # RGB red

# Green for points below threshold
color_array[mask_below] = [0.0, 1.0, 0.0]  # RGB green

# --- Step 3: Create PyVista point cloud ---
cloud = pv.PolyData(points)
cloud["RGB"] = (color_array * 255).astype(np.uint8)  # Convert to 0–255

# --- Step 4: Visualize ---
plotter = pv.Plotter()
plotter.add_points(cloud, scalars="RGB", rgb=True, point_size=5, render_points_as_spheres=True)
plotter.add_axes()
plotter.show(window_size=[800, 800])

d:\Anaconda\envs\PointCloud\Lib\site-packages\pyvista\core\utilities\points.py:77: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:50275/index.html?ui=P_0x26548aa6590_22&reconnect=auto" class="pyvi…

In [7]:
# --- Load LAS file ---
las_path = r'PointClouds\IntermediateFile_B\B_non_ground.las'
las_path = r'PointClouds\offset_PartB.las'
las = laspy.read(las_path)

# --- Extract and filter points based on Z value ---
x = las.X
y = las.Y
z = las.Z 

mask = z >= 2500.0  # Keep points with Z >= 10

x_f = x[mask]
y_f = y[mask]
z_f = z[mask]

# --- Create a new LAS header ---
header = laspy.LasHeader(point_format=las.header.point_format, version=las.header.version)
header.scales = las.header.scales
header.offsets = las.header.offsets

# --- Create new LAS object with filtered points ---
filtered_las = laspy.LasData(header)
# filtered_las.X = ((x_f - header.offsets[0]) / header.scales[0]).astype(np.int32)
# filtered_las.Y = ((y_f - header.offsets[1]) / header.scales[1]).astype(np.int32)
# filtered_las.Z = ((z_f - header.offsets[2]) / header.scales[2]).astype(np.int32)
filtered_las.X = x_f.astype(np.int32)
filtered_las.Y = y_f.astype(np.int32)
filtered_las.Z = z_f.astype(np.int32)

# Optional: copy additional attributes
if 'intensity' in las.point_format.dimension_names:
    filtered_las.intensity = las.intensity[mask]
if 'classification' in las.point_format.dimension_names:
    filtered_las.classification = las.classification[mask]

# --- Save filtered point cloud ---
output_path = r'PointClouds\IntermediateFile_B\B_non_ground_filtered.las'
filtered_las.write(output_path)
print(f"Saved filtered LAS to: {output_path}")

Saved filtered LAS to: PointClouds\IntermediateFile_B\B_non_ground_filtered.las
